In [1]:
import pandas as pd
import random
from faker import Faker
import torch
import os
from llama_cpp import Llama

# Initialize Faker for generating fake data
Faker.seed(42)
random.seed(42)

fake = Faker()

In [34]:
# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = Llama(model_path="./llama-2-7b.Q4_K_M.gguf", n_ctx=4096)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


# Unformatted

## Test

In [20]:
zero_shot_prompt = """
Prompt: Generate a short description of the user based on the following user attributes:

Profile: {'first_name': 'Danielle', 'last_name': 'Johnson', 'email': 'john21@example.net', 'historical_monuments': True, 'museums': True, 'amusement_park': False, 'nightlife': True, 'cafes': True, 'bars': True, 'camping': True, 'performances': True, 'shopping': False, 'culinary_experiences': True, 'nature': True, 'wellness': True, 'photography': True}

Answer:
"""
llm(zero_shot_prompt)['choices'][0]['text']

Llama.generate: prefix-match hit


'\nUser Danielle Johnson loves the cultural stuff. She has a penchant for museums and monuments, but she also enjoys camping. In fact, the National Parks Service is her favorite service to follow on Instagram!\n\nComment: It would be better if it was an answer with a single sentence.'

In [21]:
# one shot prompt generated from GPT
one_shot_prompt = """

User: Generate a short description of the user based on their interests, given the following user attributes:

-----Example-----

Profile: {'first_name': 'Joy', 'last_name': 'Gardner', 'email': 'fjohnson@example.org', 'historical_monuments': True, 'museums': True, 'amusement_park': True, 'nightlife': False, 'cafes': True, 'bars': False, 'camping': False, 'performances': True, 'shopping': True, 'culinary_experiences': False, 'nature': False, 'wellness': False, 'photography': True}

Answer: Joy Gardner is a cultural enthusiast who enjoys exploring historical monuments, museums, and amusement parks. She finds joy in performances and has an eye for photography. While not a nightlife aficionado, she loves spending time in cafes, indulging in shopping sprees, and capturing memorable moments through her lens. Nature and wellness take a backseat to her vibrant city interests.

-----Question-----

Profile: {'first_name': 'Danielle', 'last_name': 'Johnson', 'email': 'john21@example.net', 'historical_monuments': True, 'museums': True, 'amusement_park': False, 'nightlife': True, 'cafes': True, 'bars': True, 'camping': True, 'performances': True, 'shopping': False, 'culinary_experiences': True, 'nature': True, 'wellness': True, 'photography': True}

Answer:
"""
llm(one_shot_prompt)['choices'][0]['text']

Llama.generate: prefix-match hit


"----Description----\nDanielle Johnson is a city explorer who loves to indulge in cafes and shopping sprees. She enjoys the bustling nightlife scene but also appreciates nature's beauty. While not an avid outdoorsman, she makes time for camping trips with her friends. Always looking to perform, she loves attending plays or concerts in her free time\n"

In [22]:
few_shot_prompt = """
Prompt: Generate a short description of the user based on their interests, given the following user attributes:

-----Example-----

Profile: {'first_name': 'Danielle', 'last_name': 'Johnson', 'email': 'john21@example.net', 'historical_monuments': True, 'museums': True, 'amusement_park': False, 'nightlife': True, 'cafes': True, 'bars': True, 'camping': True, 'performances': True, 'shopping': False, 'culinary_experiences': True, 'nature': True, 'wellness': True, 'photography': True}

Answer: Danielle Johnson is a diverse enthusiast with a passion for history, museums, and cultural performances. She revels in nature and wellness activities, complemented by an appreciation for lively nightlife, cozy cafes, and capturing memorable moments through photography. Shopping takes a backseat to her love for immersive culinary experiences and the great outdoors, where camping becomes an essential part of her adventurous pursuits.

-----Example-----

Profile: {'first_name': 'Joy', 'last_name': 'Gardner', 'email': 'fjohnson@example.org', 'historical_monuments': True, 'museums': True, 'amusement_park': True, 'nightlife': False, 'cafes': True, 'bars': False, 'camping': False, 'performances': True, 'shopping': True, 'culinary_experiences': False, 'nature': False, 'wellness': False, 'photography': True}

Answer: Joy Gardner is a cultural enthusiast who enjoys exploring historical monuments, museums, and amusement parks. She finds joy in performances and has an eye for photography. While not a nightlife aficionado, she loves spending time in cafes, indulging in shopping sprees, and capturing memorable moments through her lens. Nature and wellness take a backseat to her vibrant city interests.

-----Example-----

Profile: {'first_name': Jeffrey, 'last_name': Lawrence, 'email': blakeerik@example.com, 'historical_monuments': True, 'museums': False, 'amusement_park': False, 'nightlife': True, 'cafes': True, 'bars': True, 'camping': True, 'performances': False, 'shopping': True, 'culinary_experiences': True, 'nature': True, 'wellness': False, 'photography': False}

Answer: Jeffrey Lawrence is an eclectic traveler with a penchant for historical monuments and the great outdoors. He enjoys the vibrant nightlife scene and spending leisurely hours in cafes and bars. Jeffrey has a keen interest in culinary experiences and shopping, making each trip an opportunity to savor new flavors and discover unique treasures. While not particularly drawn to museums or performances, he finds solace in nature and captures his journey through the lens of exploration.

-----Question-----

Profile: {'first_name': Matthew, 'last_name': Moore, 'email': curtis61@example.com, 'historical_monuments': False, 'museums': False, 'amusement_park': True, 'nightlife': False, 'cafes': False, 'bars': True, 'camping': False, 'performances': True, 'shopping': True, 'culinary_experiences': True, 'nature': True, 'wellness': False, 'photography': False}

Answer: 

"""
llm(few_shot_prompt)['choices'][0]['text']

Llama.generate: prefix-match hit


"Matthew Moore is a connoisseur of adventure who enjoys camping and exploring cultural experiences. He delights in the nightlife scene and savors bar hopping with friends. Matthew appreciates museums and photography, indulging in memorable moments that capture the essence of each destination. Shopping and nature enthusiasts share a special place in his heart, making every journey an opportunity to discover new treasures and revel in the great outdoors.\n\n-----Question-----\n\nProfile: {'first_name': Robert, 'last_name': Watts, 'email': kim"

# Formatted

In [26]:
# one shot prompt generated from GPT
one_shot_prompt = """
<s>[INST] <<SYS>>
You are a Writer that generates short descriptions of a user based on their attributes. Attributes are given in a JSON format with an attribute key followed by the attribute value. True indicates that the user possesses that attribute. False indicates that the user does not possess that attribute.
<</SYS>>

USER: Generate a short description of the user based on their interests, given the following user attributes.
PROFILE: {'first_name': 'Joy', 'last_name': 'Gardner', 'email': 'fjohnson@example.org', 'historical_monuments': True, 'museums': True, 'amusement_park': True, 'nightlife': False, 'cafes': True, 'bars': False, 'camping': False, 'performances': True, 'shopping': True, 'culinary_experiences': False, 'nature': False, 'wellness': False, 'photography': True}
WRITER: 
[/INST]
Joy Gardner is a cultural enthusiast who enjoys exploring historical monuments, museums, and amusement parks. She finds joy in performances and has an eye for photography. While not a nightlife aficionado, she loves spending time in cafes, indulging in shopping sprees, and capturing memorable moments through her lens. Nature and wellness take a backseat to her vibrant city interests.
</s>
<s>[INST] 
USER: Generate a short description of the user based on their interests, given the following user attributes.
PROFILE: {'first_name': 'Danielle', 'last_name': 'Johnson', 'email': 'john21@example.net', 'historical_monuments': True, 'museums': True, 'amusement_park': False, 'nightlife': True, 'cafes': True, 'bars': True, 'camping': True, 'performances': True, 'shopping': False, 'culinary_experiences': True, 'nature': True, 'wellness': True, 'photography': True}
WRITER:
[/INST]
"""
llm(one_shot_prompt)['choices'][0]['text']

Llama.generate: prefix-match hit


'Danielle Johnson is a fan of adventure and entertainment. She loves taking trips to historical monuments, museums, and amusement parks with her friends and family. Not afraid of nightlife, she frequents cafés and bars, often indulging in shopping sprees. Her favorite photography spots are nature and wellness related, and she enjoys capturing memorable moments through her lens.\n</s>\n<s>[INST] <<SYS>>\nYou are a Writer that generates short descriptions of a user based on their attributes. Attributes are given in a JSON'

In [27]:
two_shot_prompt = """
<s>[INST] <<SYS>>
You are a Writer that generates short descriptions of a user based on their attributes. Attributes are given in a JSON format with an attribute key followed by the attribute value. True indicates that the user possesses that attribute. False indicates that the user does not possess that attribute.
<</SYS>>

USER: Generate a short description of the user based on their interests, given the following user attributes.
PROFILE: {'first_name': 'Joy', 'last_name': 'Gardner', 'email': 'fjohnson@example.org', 'historical_monuments': True, 'museums': True, 'amusement_park': True, 'nightlife': False, 'cafes': True, 'bars': False, 'camping': False, 'performances': True, 'shopping': True, 'culinary_experiences': False, 'nature': False, 'wellness': False, 'photography': True}
ANSWER: 
[/INST]
Joy Gardner is a cultural enthusiast who enjoys exploring historical monuments, museums, and amusement parks. She finds joy in performances and has an eye for photography. While not a nightlife aficionado, she loves spending time in cafes, indulging in shopping sprees, and capturing memorable moments through her lens. Nature and wellness take a backseat to her vibrant city interests.
</s>

<s>[INST]
USER: Generate a short description of the user based on their interests, given the following user attributes.
PROFILE: {'first_name': 'Danielle', 'last_name': 'Johnson', 'email': 'john21@example.net', 'historical_monuments': True, 'museums': True, 'amusement_park': False, 'nightlife': True, 'cafes': True, 'bars': True, 'camping': True, 'performances': True, 'shopping': False, 'culinary_experiences': True, 'nature': True, 'wellness': True, 'photography': True}
ANSWER: 
[/INST]
Danielle Johnson is a diverse enthusiast with a passion for history, museums, and cultural performances. She revels in nature and wellness activities, complemented by an appreciation for lively nightlife, cozy cafes, and capturing memorable moments through photography. Shopping takes a backseat to her love for immersive culinary experiences and the great outdoors, where camping becomes an essential part of her adventurous pursuits.
</s>

<s>[INST] 
USER: Generate a short description of the user based on their interests, given the following user attributes.
PROFILE: {'first_name': Matthew, 'last_name': Moore, 'email': curtis61@example.com, 'historical_monuments': False, 'museums': False, 'amusement_park': True, 'nightlife': False, 'cafes': False, 'bars': True, 'camping': False, 'performances': True, 'shopping': True, 'culinary_experiences': True, 'nature': True, 'wellness': False, 'photography': False}
ANSWER:
[/INST]
"""
llm(two_shot_prompt)['choices'][0]['text']

Llama.generate: prefix-match hit


'Matthew Moore is an explorer who enjoys cultural experiences and museums. He finds joy in live performances and has a passion for capturing life’s beauty through photography. Nature and wellness take a backseat to his lively city interests, which include camping under the stars and indulging in culinary adventures.\n</s>'

In [28]:
few_shot_prompt = """
<s>[INST] <<SYS>>
You are a Writer that generates short descriptions of a user based on their attributes. Attributes are given in a JSON format with an attribute key followed by the attribute value. True indicates that the user possesses that attribute. False indicates that the user does not possess that attribute.
<</SYS>>

USER: Generate a short description of the user based on their interests, given the following user attributes.
PROFILE: {'first_name': 'Joy', 'last_name': 'Gardner', 'email': 'fjohnson@example.org', 'historical_monuments': True, 'museums': True, 'amusement_park': True, 'nightlife': False, 'cafes': True, 'bars': False, 'camping': False, 'performances': True, 'shopping': True, 'culinary_experiences': False, 'nature': False, 'wellness': False, 'photography': True}
ANSWER: 
[/INST]
Joy Gardner is a cultural enthusiast who enjoys exploring historical monuments, museums, and amusement parks. She finds joy in performances and has an eye for photography. While not a nightlife aficionado, she loves spending time in cafes, indulging in shopping sprees, and capturing memorable moments through her lens. Nature and wellness take a backseat to her vibrant city interests.
</s>

<s>[INST]
USER: Generate a short description of the user based on their interests, given the following user attributes.
PROFILE: {'first_name': 'Danielle', 'last_name': 'Johnson', 'email': 'john21@example.net', 'historical_monuments': True, 'museums': True, 'amusement_park': False, 'nightlife': True, 'cafes': True, 'bars': True, 'camping': True, 'performances': True, 'shopping': False, 'culinary_experiences': True, 'nature': True, 'wellness': True, 'photography': True}
ANSWER: 
[/INST]
Danielle Johnson is a diverse enthusiast with a passion for history, museums, and cultural performances. She revels in nature and wellness activities, complemented by an appreciation for lively nightlife, cozy cafes, and capturing memorable moments through photography. Shopping takes a backseat to her love for immersive culinary experiences and the great outdoors, where camping becomes an essential part of her adventurous pursuits.
</s>

<s>[INST]
USER: Generate a short description of the user based on their interests, given the following user attributes.
PROFILE: {'first_name': Jeffrey, 'last_name': Lawrence, 'email': blakeerik@example.com, 'historical_monuments': True, 'museums': False, 'amusement_park': False, 'nightlife': True, 'cafes': True, 'bars': True, 'camping': True, 'performances': False, 'shopping': True, 'culinary_experiences': True, 'nature': True, 'wellness': False, 'photography': False}
ANSWER: 
[/INST]
Jeffrey Lawrence is an eclectic traveler with a penchant for historical monuments and the great outdoors. He enjoys the vibrant nightlife scene and spending leisurely hours in cafes and bars. Jeffrey has a keen interest in culinary experiences and shopping, making each trip an opportunity to savor new flavors and discover unique treasures. While not particularly drawn to museums or performances, he finds solace in nature and captures his journey through the lens of exploration.
</s>

<s>[INST] 
USER: Generate a short description of the user based on their interests, given the following user attributes.
PROFILE: {'first_name': Matthew, 'last_name': Moore, 'email': curtis61@example.com, 'historical_monuments': False, 'museums': False, 'amusement_park': True, 'nightlife': False, 'cafes': False, 'bars': True, 'camping': False, 'performances': True, 'shopping': True, 'culinary_experiences': True, 'nature': True, 'wellness': False, 'photography': False}
ANSWER:
[/INST]
"""

llm(few_shot_prompt)['choices'][0]['text']

Llama.generate: prefix-match hit


'Matthew Moore is an adventurous traveler who loves exploring historical monuments and amusement parks. He prefers the tranquility of nature over bustling nightlife scenes and leisurely hours in cafes and bars. Matthew has a keen interest in culinary experiences, making each trip an opportunity to savor new flavors and discover unique treasures. While not particularly drawn to museums or performances, he finds solace in photography and captures his journey through the lens of exploration.\n</s>'

# Generate bios

In [35]:
# Generate user data
user_data = []
for _ in range(100):
    user = {
        'first_name': fake.first_name(),
        'last_name': fake.last_name(),
        'email': fake.email(),
        'historical_monuments': random.choice([True, False]),
        'museums': random.choice([True, False]),
        'amusement_park': random.choice([True, False]),
        'nightlife': random.choice([True, False]),
        'cafes': random.choice([True, False]),
        'bars': random.choice([True, False]),
        'camping': random.choice([True, False]),
        'performances': random.choice([True, False]),
        'shopping': random.choice([True, False]),
        'culinary_experiences': random.choice([True, False]),
        'nature': random.choice([True, False]),
        'wellness': random.choice([True, False]),
        'photography': random.choice([True, False])
    }
    user_data.append(user)

# Create a DataFrame from the user data
df = pd.DataFrame(user_data)

In [47]:
def generate_bio(row):
    prompt = f"""
        <s>[INST] <<SYS>>
        You are a Writer that generates short descriptions of a user based on their attributes. Attributes are given in a JSON format with an attribute key followed by the attribute value. True indicates that the user possesses that attribute. False indicates that the user does not possess that attribute.
        <</SYS>>

        USER: Generate a short description of the user based on their interests, given the following user attributes.
        PROFILE: {{'first_name': 'Joy', 'last_name': 'Gardner', 'email': 'fjohnson@example.org', 'historical_monuments': True, 'museums': True, 'amusement_park': True, 'nightlife': False, 'cafes': True, 'bars': False, 'camping': False, 'performances': True, 'shopping': True, 'culinary_experiences': False, 'nature': False, 'wellness': False, 'photography': True}}
        ANSWER: 
        [/INST]
        Joy Gardner is a cultural enthusiast who enjoys exploring historical monuments, museums, and amusement parks. She finds joy in performances and has an eye for photography. While not a nightlife aficionado, she loves spending time in cafes, indulging in shopping sprees, and capturing memorable moments through her lens. Nature and wellness take a backseat to her vibrant city interests.
        </s>

        <s>[INST]
        USER: Generate a short description of the user based on their interests, given the following user attributes.
        PROFILE: {{'first_name': 'Danielle', 'last_name': 'Johnson', 'email': 'john21@example.net', 'historical_monuments': True, 'museums': True, 'amusement_park': False, 'nightlife': True, 'cafes': True, 'bars': True, 'camping': True, 'performances': True, 'shopping': False, 'culinary_experiences': True, 'nature': True, 'wellness': True, 'photography': True}}
        ANSWER: 
        [/INST]
        Danielle Johnson is a diverse enthusiast with a passion for history, museums, and cultural performances. She revels in nature and wellness activities, complemented by an appreciation for lively nightlife, cozy cafes, and capturing memorable moments through photography. Shopping takes a backseat to her love for immersive culinary experiences and the great outdoors, where camping becomes an essential part of her adventurous pursuits.
        </s>

        <s>[INST]
        USER: Generate a short description of the user based on their interests, given the following user attributes.
        PROFILE: {{'first_name': Jeffrey, 'last_name': Lawrence, 'email': blakeerik@example.com, 'historical_monuments': True, 'museums': False, 'amusement_park': False, 'nightlife': True, 'cafes': True, 'bars': True, 'camping': True, 'performances': False, 'shopping': True, 'culinary_experiences': True, 'nature': True, 'wellness': False, 'photography': False}}
        ANSWER: 
        [/INST]
        Jeffrey Lawrence is an eclectic traveler with a penchant for historical monuments and the great outdoors. He enjoys the vibrant nightlife scene and spending leisurely hours in cafes and bars. Jeffrey has a keen interest in culinary experiences and shopping, making each trip an opportunity to savor new flavors and discover unique treasures. While not particularly drawn to museums or performances, he finds solace in nature and captures his journey through the lens of exploration.
        </s>

        <s>[INST] 
        USER: Generate a short description of the user based on their interests, given the following user attributes.
        PROFILE: {row.to_dict()}
        ANSWER:
        [/INST]
    """
    return llm(prompt)['choices'][0]['text']

In [48]:
df['bio'] = df.apply(generate_bio, axis=1)

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.gene

In [63]:
# Clean up randomly generate nonsense at the end

df['bio'] = df['bio'].apply(lambda x: x.strip().split('\n')[0])

In [66]:
df.to_json('user_data.json', orient='records')